In [1]:
!python --version
!pip install langchain langchain-openai langchain-anthropic langgraph
!pip install transformers==4.51.3 peft==0.13.2 safetensors==0.4.3
!pip install langchain_huggingface

Python 3.12.12
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 453.7/453.7 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5 kB 50.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.2.12
    Uninstalling langchain-core-1.2.12:
      Successfully uninstalled langchain-core-1.2.12
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 113.6 MB/s eta 0:00:00
  Attempting uninstall: safetensors
    Found existing installation: safetensors 0.7

In [2]:
# imports for HuggingFace LLM Loading

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFacePipeline

In [3]:
# Download Kanana-1.5 2.1B Base Original Model

!huggingface-cli download kakaocorp/kanana-1.5-2.1b-base --local-dir kakao_original

⚠️  Warning: 'huggingface-cli download' is deprecated. Use 'hf download' instead.
Fetching 13 files:   0% 0/13 [00:00<?, ?it/s]Downloading 'README.md' to 'kakao_original/.cache/huggingface/download/Xn7B-BWUGOee2Y6hCZtEhtFu4BE=.8e3fb650257b06def9d43a3a2d23e3e12d050968.incomplete'

README.md: 0.00B [00:00, ?B/s]

README.md: 6.05kB [00:00, 1.80MB/s]
Download complete. Moving file to kakao_original/README.md
LICENSE: 10.5kB [00:00, 3.09MB/s]
Download complete. Moving file to kakao_original/LICENSE

.gitattributes: 1.60kB [00:00, 803kB/s]
Download complete. Moving file to kakao_original/.gitattributes
Fetching 13 files:   8% 1/13 [00:00<00:02,  4.79it/s]
kanana-logo.png:   0% 0.00/59.0k [00:00<?, ?B/s]

kanana-logo-dark.png:   0% 0.00/59.1k [00:00<?, ?B/s]


kanana-logo.png: 100% 59.0k/59.0k [00:00<00:00, 7.27MB/s]
Download complete. Moving file to kakao_original/assets/logo/kanana-logo.png
kanana-logo-dark.png: 100% 59.1k/59.1k [00:00<00:00, 11.3MB/s]
Download complete. Moving file to kaka

In [4]:
# load Oh-LoRA original LLM

import os
print(os.path.abspath('ohlora_llm'))

llm_path = 'ohlora_llm'
ohlora_llm = AutoModelForCausalLM.from_pretrained(
    llm_path,
    trust_remote_code=True,
    torch_dtype=torch.float16
)
tokenizer = AutoTokenizer.from_pretrained(llm_path)

pipe = pipeline(
    "text-generation",
    model=ohlora_llm,
    tokenizer=tokenizer,
    max_new_tokens=64,
    temperature=0.6,
    top_p=0.95
)

/content/ohlora_llm


Device set to use cuda:0


In [11]:
# convert to LangChain LLM

local_llm = HuggingFacePipeline(pipeline=pipe)

**LangGraph 기반 메모리 구현**

In [12]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import MemorySaver

In [18]:
# create memory saver
checkpointer = MemorySaver()

# create agent
agent = create_agent(
    local_llm,
    tools=[],
    checkpointer=checkpointer
)

# config
config = {"configurable": {"thread_id": "conversation-1"}}

In [20]:
# 1번째 대화

result1 = agent.invoke(
    {"messages": [
        {"role": "user",
         "content": "로라야 나 장원영 좋아해. 너도? (답변 시작)"}
    ]},
    config=config
)
result1_answer = result1["messages"][-1].content
result1_answer_final = result1_answer.split('(답변 시작)')[2].split('(답변 종료)')[0]

result1_answer_final

/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


' 나도 장원영 좋아해! 완전 럭키비키라는 말 알아? 🍀 '

In [21]:
# 2번째 대화

result2 = agent.invoke(
    {"messages": [
        {"role": "user",
         "content": "로라야 내가 누구 좋아한다고 했지? (답변 시작)"}
    ]},
    config=config
)
result2_answer = result2["messages"][-1].content
result2_answer_final = result2_answer.split('(답변 시작)')[2].split('(답변 종료)')[0]

result2_answer_final

/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


' 나도 장원영 좋아해! 완전 럭키비키라는 말 알아? 🍀 '